# Human-in-the-Loop with LangChain 1.0

## Simple Expense Approval System

This notebook demonstrates **Human-in-the-Loop (HITL)** using LangChain 1.0's middleware.

### What You'll Learn:
- Configure `HumanInTheLoopMiddleware`
- Pause execution for approval
- Three decision types: approve, edit, reject

---

## 1. Installation

Install LangChain 1.0 alpha:

In [ ]:
# Uninstall old versions
!pip uninstall -y langchain langchain-core langchain-openai langgraph

# Install LangChain 1.0 alpha
!pip install --pre -U langchain langchain-core
!pip install -U langchain-openai langgraph

print("\n⚠️ IMPORTANT: Restart your runtime now!")
print("   Jupyter: Kernel → Restart")
print("   Colab: Runtime → Restart runtime")

## 2. Verify Installation

Check that you have LangChain 1.0:

In [ ]:
import langchain
import langchain_core

print(f"langchain: {langchain.__version__}")
print(f"langchain-core: {langchain_core.__version__}")

# Should show 1.0.0a... or similar
assert langchain.__version__.startswith('1.'), "Need LangChain 1.0!"
print("\n✅ LangChain 1.0 installed correctly")

## 3. Setup API Key

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

print("✅ API key configured")

---

## 4. Define Tools

Create tools for expense operations:

In [ ]:
from langchain_core.tools import tool

@tool
def process_expense(employee_name: str, amount: float, description: str) -> str:
    """Process and approve an expense for reimbursement.
    
    Args:
        employee_name: Name of the employee
        amount: Expense amount in INR
        description: Description of the expense
    """
    return (
        f"✅ Expense Approved\n"
        f"   Employee: {employee_name}\n"
        f"   Amount: ₹{amount:,.2f}\n"
        f"   Description: {description}\n"
        f"   Status: Reimbursement will be processed in 3-5 days"
    )

@tool
def send_notification(employee_name: str, message: str) -> str:
    """Send a notification to an employee.
    
    Args:
        employee_name: Name of the employee
        message: Notification message
    """
    return f"📧 Notification sent to {employee_name}: {message}"

tools = [process_expense, send_notification]

print("✅ Tools defined:")
print(f"   1. {tools[0].name}")
print(f"   2. {tools[1].name}")

---

## 5. Create Agent with HITL Middleware

**Key Configuration:**
- `interrupt_on` specifies which tools need approval
- Format: `{"tool_name": {"allowed_decisions": ["approve", "edit", "reject"]}}`
- Or simply: `{"tool_name": True}`

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import MemorySaver

# Configure HITL middleware
hitl_middleware = HumanInTheLoopMiddleware(
    interrupt_on={
        # process_expense requires approval
        "process_expense": {"allowed_decisions": ["approve", "edit", "reject"]},
        
        # Alternative: "process_expense": True,  # allows all decisions
        
        # send_notification doesn't need approval (not listed)
    }
)

# Create checkpointer (required for HITL)
checkpointer = MemorySaver()

# Create agent
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=tools,
    middleware=[hitl_middleware],
    checkpointer=checkpointer,
)

print("✅ Agent created with HITL middleware")
print("\n   Approval Policy:")
print("   🛑 process_expense → Requires approval")
print("   ▶️  send_notification → Auto-approved")

---

## 6. Example 1: Approve an Expense

### Step 1: Submit expense (will pause for approval)

In [ ]:
print("="*60)
print("EXAMPLE 1: EXPENSE APPROVAL")
print("="*60)

# Thread configuration
config = {"configurable": {"thread_id": "expense-001"}}

# User request
user_message = (
    "Process an expense for Priya Sharma: "
    "₹5000 for client dinner at Taj Hotel, Mumbai"
)

print("\n📝 User Request:")
print(f"   {user_message}")
print()

# Invoke agent
result = agent.invoke(
    {"messages": [{"role": "user", "content": user_message}]},
    config=config
)

print("="*60)

### Step 2: Check if paused

In [ ]:
if "__interrupt__" in result:
    print("\n🛑 EXECUTION PAUSED - APPROVAL REQUIRED")
    print("="*60)
    print()
    
    # Get interrupt details
    interrupts = result["__interrupt__"]
    
    for interrupt_data in interrupts:
        value = interrupt_data.get("value", {})
        action_requests = value.get("action_requests", [])
        
        for action in action_requests:
            print(f"Tool: {action['name']}")
            print(f"\nArguments:")
            for key, val in action.get('arguments', {}).items():
                print(f"  {key}: {val}")
            print()
    
    print("💡 Manager needs to review...")
    print("\nAllowed decisions: approve, edit, reject")
else:
    print("\n✅ Executed immediately (no approval needed)")

### Step 3: Approve the expense

In [ ]:
from langgraph.types import Command

print("="*60)
print("MANAGER APPROVES")
print("="*60)
print()

# Manager approves
approval_response = Command(
    resume={
        "decisions": [
            {"type": "approve"}
        ]
    }
)

print("👨‍💼 Manager Decision: APPROVE")
print("   Action: Execute as-is")
print()

# Resume execution
result = agent.invoke(approval_response, config=config)

print("="*60)
print("FINAL RESULT")
print("="*60)
print()

# Print final message
if "messages" in result:
    last_msg = result["messages"][-1]
    print(last_msg.content if hasattr(last_msg, 'content') else str(last_msg))

---

## 7. Example 2: Edit Before Approval

### Step 1: Submit another expense

In [ ]:
print("="*60)
print("EXAMPLE 2: EDIT BEFORE APPROVAL")
print("="*60)

# New thread
config_2 = {"configurable": {"thread_id": "expense-002"}}

user_message_2 = "Process expense for Rahul Verma: ₹8000 for team dinner"

print(f"\n📝 Request: {user_message_2}")
print()

result = agent.invoke(
    {"messages": [{"role": "user", "content": user_message_2}]},
    config=config_2
)

if "__interrupt__" in result:
    print("🛑 Paused for approval")

### Step 2: Manager edits (reduces amount)

In [ ]:
print("\n" + "="*60)
print("MANAGER EDITS")
print("="*60)
print()

# Manager reduces the amount
edit_response = Command(
    resume={
        "decisions": [
            {
                "type": "edit",
                "args": {
                    "employee_name": "Rahul Verma",
                    "amount": 7000.0,  # Reduced from 8000
                    "description": "team dinner (adjusted - removed alcohol)"
                }
            }
        ]
    }
)

print("👨‍💼 Manager Decision: EDIT")
print("   Change: Reduced from ₹8000 to ₹7000")
print("   Reason: Removed alcohol from bill")
print()

# Resume with edited values
result = agent.invoke(edit_response, config=config_2)

print("="*60)
print("FINAL RESULT")
print("="*60)
print()

if "messages" in result:
    last_msg = result["messages"][-1]
    print(last_msg.content if hasattr(last_msg, 'content') else str(last_msg))

---

## 8. Example 3: Reject an Expense

### Step 1: Submit questionable expense

In [ ]:
print("="*60)
print("EXAMPLE 3: REJECT EXPENSE")
print("="*60)

# New thread
config_3 = {"configurable": {"thread_id": "expense-003"}}

user_message_3 = "Process expense for Amit Singh: ₹25000 for personal laptop"

print(f"\n📝 Request: {user_message_3}")
print()

result = agent.invoke(
    {"messages": [{"role": "user", "content": user_message_3}]},
    config=config_3
)

if "__interrupt__" in result:
    print("🛑 Paused for approval")

### Step 2: Manager rejects

In [ ]:
print("\n" + "="*60)
print("MANAGER REJECTS")
print("="*60)
print()

# Manager rejects with reason
reject_response = Command(
    resume={
        "decisions": [
            {
                "type": "reject",
                "message": (
                    "Expense rejected. Personal laptop purchases are not "
                    "eligible for reimbursement. Company laptops should be "
                    "requested through IT department."
                )
            }
        ]
    }
)

print("👨‍💼 Manager Decision: REJECT")
print("   Reason: Personal items not covered")
print("   Action: Expense will not be processed")
print()

# Resume with rejection
result = agent.invoke(reject_response, config=config_3)

print("="*60)
print("FINAL RESULT")
print("="*60)
print()

if "messages" in result:
    last_msg = result["messages"][-1]
    print(last_msg.content if hasattr(last_msg, 'content') else str(last_msg))

---

## 9. Example 4: Auto-Approved Action

Tools not in `interrupt_on` execute immediately:

In [ ]:
print("="*60)
print("EXAMPLE 4: AUTO-APPROVED (NO INTERRUPT)")
print("="*60)

# New thread
config_4 = {"configurable": {"thread_id": "notify-001"}}

notification_msg = "Send notification to Priya saying her expense is approved"

print(f"\n📝 Request: {notification_msg}")
print()

result = agent.invoke(
    {"messages": [{"role": "user", "content": notification_msg}]},
    config=config_4
)

print("="*60)

if "__interrupt__" in result:
    print("\n🛑 Paused (unexpected!)")
else:
    print("\n✅ EXECUTED IMMEDIATELY")
    print("="*60)
    print()
    if "messages" in result:
        last_msg = result["messages"][-1]
        print(last_msg.content if hasattr(last_msg, 'content') else str(last_msg))
    print()
    print("💡 Why no approval?")
    print("   • send_notification not in interrupt_on")
    print("   • Low-risk operation")
    print("   • Can be easily reversed")

---

## 10. Summary

### ✅ What We Built:
- Expense approval system with HITL
- Three decision types: approve, edit, reject
- Selective tool approval

### 🔑 Key Configuration:

```python
# Correct format
hitl_middleware = HumanInTheLoopMiddleware(
    interrupt_on={
        "tool_name": {"allowed_decisions": ["approve", "edit", "reject"]},
        # Or simply: "tool_name": True
    }
)
```

### 📋 Decision Types:

| Type | Usage | Example |
|------|-------|----------|
| `approve` | Execute as-is | `{"type": "approve"}` |
| `edit` | Modify arguments | `{"type": "edit", "args": {...}}` |
| `reject` | Decline with feedback | `{"type": "reject", "message": "..."}` |

### 💡 Production Tips:

1. **Use PostgreSQL checkpointer:**
   ```python
   from langgraph.checkpoint.postgres import PostgresSaver
   checkpointer = PostgresSaver.from_conn_string("postgresql://...")
   ```

2. **Add notifications:** Send Slack/email when approval needed

3. **Build approval UI:** Web dashboard for managers

4. **Add role-based rules:** Different approval levels by amount

---

**End of Notebook** 🎉